In [1]:
import numpy as np
import pandas as pd
import math

from sklearn.metrics import accuracy_score

import igraph
import pickle

from abc import abstractmethod
import torch

In [2]:
# Source: https://github.com/thibaudmartinez/label-propagation/blob/master/notebook.ipynb

class BaseLabelPropagation:
    """Base class for label propagation models.

    Parameters
    ----------
    adj_matrix: torch.FloatTensor
        Adjacency matrix of the graph.
    """
    def __init__(self, adj_matrix):
        self.norm_adj_matrix = self._normalize(adj_matrix)
        self.n_nodes = adj_matrix.size(0)
        self.one_hot_labels = None 
        self.n_classes = None
        self.labeled_mask = None
        self.predictions = None

    @staticmethod
    @abstractmethod
    def _normalize(adj_matrix):
        raise NotImplementedError("_normalize must be implemented")

    @abstractmethod
    def _propagate(self):
        raise NotImplementedError("_propagate must be implemented")

    def _one_hot_encode(self, labels):
        # Get the number of classes
        classes = torch.unique(labels)
        classes = classes[classes != -1]
        self.n_classes = classes.size(0)

        # One-hot encode labeled data instances and zero rows corresponding to unlabeled instances
        unlabeled_mask = (labels == -1)
        labels = labels.clone()  # defensive copying
        labels[unlabeled_mask] = 0
        self.one_hot_labels = torch.zeros((self.n_nodes, self.n_classes), dtype=torch.float)
        self.one_hot_labels = self.one_hot_labels.scatter(1, labels.unsqueeze(1), 1)
        self.one_hot_labels[unlabeled_mask, 0] = 0

        self.labeled_mask = ~unlabeled_mask

    def fit(self, labels, max_iter, tol):
        """Fits a semi-supervised learning label propagation model.

        labels: torch.LongTensor
            Tensor of size n_nodes indicating the class number of each node.
            Unlabeled nodes are denoted with -1.
        max_iter: int
            Maximum number of iterations allowed.
        tol: float
            Convergence tolerance: threshold to consider the system at steady state.
        """
        self._one_hot_encode(labels)

        self.predictions = self.one_hot_labels.clone()
        prev_predictions = torch.zeros((self.n_nodes, self.n_classes), dtype=torch.float)

        for i in range(max_iter):
            # Stop iterations if the system is considered at a steady state
            variation = torch.abs(self.predictions - prev_predictions).sum().item()

            if variation < tol:
                print(f"The method stopped after {i} iterations, variation={variation:.4f}.")
                break

            prev_predictions = self.predictions
            self._propagate()

    def predict(self):
        return self.predictions

    def predict_classes(self):
        return self.predictions.max(dim=1).indices

In [3]:
class LabelPropagation(BaseLabelPropagation):
    def __init__(self, adj_matrix):
        super().__init__(adj_matrix)

    @staticmethod
    def _normalize(adj_matrix):
        """Computes D^-1 * W"""
        degs = adj_matrix.sum(dim=1)
        degs[degs == 0] = 1  # avoid division by 0 error
        return adj_matrix / degs[:, None]

    def _propagate(self):
        self.predictions = torch.matmul(self.norm_adj_matrix, self.predictions)

        # Put back already known labels
        self.predictions[self.labeled_mask] = self.one_hot_labels[self.labeled_mask]

    def fit(self, labels, max_iter=1000, tol=1e-3):
        super().fit(labels, max_iter, tol)


In [4]:
class LabelSpreading(BaseLabelPropagation):
    def __init__(self, adj_matrix):
        super().__init__(adj_matrix)
        self.alpha = None

    @staticmethod
    def _normalize(adj_matrix):
        """Computes D^-1/2 * W * D^-1/2"""
        degs = adj_matrix.sum(dim=1)
        norm = torch.pow(degs, -0.5)
        norm[torch.isinf(norm)] = 1
        return adj_matrix * norm[:, None] * norm[None, :]

    def _propagate(self):
        self.predictions = (
            self.alpha * torch.matmul(self.norm_adj_matrix, self.predictions)
            + (1 - self.alpha) * self.one_hot_labels
        )

    def fit(self, labels, max_iter=1000, tol=1e-3, alpha=0.5):
        """
        Parameters
        ----------
        alpha: float
            Clamping factor.
        """
        self.alpha = alpha
        super().fit(labels, max_iter, tol)


## Load and pre-process data

In [5]:
all_graph = pickle.load(open('./data/icwsm_polarization/all_igraph.pickle', "rb"))

In [6]:
# Preprocess graph by flipping edge if retweet. For each edge, if it is a retweet edge, flip the edge and insert it into a different graph. Along with all of the attributes.
# maintain the order.

def process_edges(graph):
    edge_attributes = graph.edge_attributes()
    attributes_dict = {attr:graph.es[attr] for attr in edge_attributes}
    
    processed_edges = []
    edgelist = graph.get_edgelist()
    for i in range(len(edgelist)):
        if (graph.es[i]['type'] == 'retweet'):
            processed_edges.append(tuple(reversed(edgelist[i])))
        else:
            processed_edges.append(edgelist[i])
    
    graph.es.delete()
    graph.add_edges(processed_edges)
    
    for attr in edge_attributes:
        graph.es[attr] = attributes_dict[attr]


In [7]:
all_graph.get_edgelist()[0:10]

[(4522, 617),
 (13126, 12049),
 (11956, 13524),
 (12875, 3983),
 (10701, 13172),
 (11427, 18252),
 (16129, 20215),
 (17900, 22096),
 (12673, 12428),
 (10815, 19042)]

In [8]:
len(all_graph.get_edgelist())

77920

In [9]:
process_edges(all_graph)

In [10]:
all_graph.get_edgelist()[0:10]

[(617, 4522),
 (12049, 13126),
 (13524, 11956),
 (3983, 12875),
 (10701, 13172),
 (18252, 11427),
 (20215, 16129),
 (17900, 22096),
 (12673, 12428),
 (19042, 10815)]

In [11]:
len(all_graph.get_edgelist()) ## will be same as previous

77920

## Generate data to give model

In [12]:
# uncomment to recompute adjacency matrix and save

# adj_matrix = np.array(all_graph.get_adjacency().data)
# np.savez_compressed("objects/adj_matrix.npz", adj_matrix)

In [13]:
# load adjacency matrix
adj_matrix = np.load("objects/adj_matrix.npz")['arr_0']

In [14]:
np.shape(adj_matrix)

(22405, 22405)

In [15]:
# Adjacency matrix is directed
adj_matrix[617][4522] != adj_matrix[4522][617]

True

In [23]:
def get_labels_subsample(graph, sample_type, proportion=0.1):
    n = len(graph.vs)
    labels = np.full(n, -1, dtype=int)
    
    n_sample = math.ceil(proportion*n)

    if sample_type=='random':
        indices = np.random.randint(0, n, size=n_sample)
    elif sample_type == 'centrality':
        indices = np.argsort(graph.pagerank())[-n_sample:]
        
    for i in indices:
        if graph.vs[i]['cluster'] == 'left':
            labels[i] = 1
        elif graph.vs[i]['cluster'] == 'right':
            labels[i] = 0
        ## if cluster is '-', leave it unlabeled (i.e. -1)
        
    return labels, indices

In [116]:
# obtain observed labels by centrality and random selection
rlabels, r_idxs = get_labels_subsample(all_graph, 'random',0.1)

In [117]:
clabels, c_idxs = get_labels_subsample(all_graph, 'centrality',0.1)

In [118]:
clabels[c_idxs[0]]

0

In [119]:
all_graph.vs[c_idxs[0]]['cluster'] # corresponds correctly to revealed labels

'right'

## Learning

In [120]:
# Create input tensors
adj_matrix_t = torch.FloatTensor(adj_matrix)

clabels_t = torch.LongTensor(clabels)
rlabels_t = torch.LongTensor(rlabels)


The method stopped after 42 iterations, variation=0.0008.
The method stopped after 45 iterations, variation=0.0009.


In [ ]:
# For labels selected based on centrality:

# Learn with Label Propagation
label_propagation_central = LabelPropagation(adj_matrix_t)
label_propagation_central.fit(clabels_t)
label_propagation_output_labels_central = label_propagation_central.predict_classes()

# Learn with Label Spreading
label_spreading_central = LabelSpreading(adj_matrix_t)
label_spreading_central.fit(clabels_t, alpha=0.8)
label_spreading_output_labels_central = label_spreading_central.predict_classes()

In [121]:
# For labels selected randomly:

# Learn with Label Propagation
label_propagation_random = LabelPropagation(adj_matrix_t)
label_propagation_random.fit(rlabels_t)
label_propagation_output_labels_random = label_propagation_random.predict_classes()

# Learn with Label Spreading
label_spreading_random = LabelSpreading(adj_matrix_t)
label_spreading_random.fit(rlabels_t, alpha=0.8)
label_spreading_output_labels_random = label_spreading_random.predict_classes()

The method stopped after 71 iterations, variation=0.0008.
The method stopped after 37 iterations, variation=0.0009.


In [122]:
# Predicted labels
label_spreading_output_labels_central.numpy()

array([0, 0, 1, ..., 1, 0, 0])

In [123]:
label_spreading_output_labels_random.numpy()

array([0, 0, 0, ..., 1, 0, 0])

## Evaluation

In [124]:
# if true label for a node is '-', remove it  

true_labels = pd.DataFrame(data=all_graph.vs["cluster"], columns=["label"])
label.head()

cleaned_true_labels = true_labels.loc[true_labels["label"] != '-']
y = cleaned_labels.label.map({"left": 1, "right": 0})

### Using label spreading

In [125]:
central_spreading_df = pd.DataFrame(data=label_spreading_output_labels_central.numpy(), columns=["label"])
cleaned_central_spreading_df = central_spreading_df.loc[label["label"] != '-']
cleaned_central_spreading_df.head()

,label
0,0
1,0
2,1
3,1
5,1


In [126]:
random_spreading_df = pd.DataFrame(data=label_spreading_output_labels_random.numpy(), columns=["label"])
cleaned_random_spreading_df = random_spreading_df.loc[label["label"] != '-']
cleaned_random_spreading_df.head()

,label
0,0
1,0
2,0
3,1
5,1


In [127]:
accuracy_score(y, cleaned_central_spreading_df.label)

0.805468327016784

In [128]:
accuracy_score(y, cleaned_random_spreading_df.label)

0.7672441797509475

### Using label propagation

In [130]:
central_propagation_df = pd.DataFrame(data=label_propagation_output_labels_central.numpy(), columns=["label"])
cleaned_central_propagation_df = central_propagation_df.loc[label["label"] != '-']
cleaned_central_propagation_df.head()

,label
0,0
1,0
2,1
3,1
5,1


In [131]:
# remove '-' labels
random_propagation_df = pd.DataFrame(data=label_propagation_output_labels_random.numpy(), columns=["label"])
cleaned_random_propagation_df = random_propagation_df.loc[label["label"] != '-']
cleaned_random_propagation_df.head()

,label
0,0
1,0
2,0
3,1
5,1


In [132]:
accuracy_score(y, cleaned_central_propagation_df.label)

0.9132647536545749

In [133]:
accuracy_score(y, cleaned_random_propagation_df.label)

0.765836491608013